In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import time

Set random seed for reproducibility

In [ ]:
torch.manual_seed(42)
np.random.seed(42)

Device configuration

In [ ]:
device = "mps"
print(f"Using device: {device}")

Load and prepare data

In [ ]:
data = pd.read_csv("./data/rnn_full_data.csv")

Create a unique location identifier and time key

In [ ]:
data["location_id"] = (
    data["Location Group"].astype(str) + "_" + data["District"].astype(str)
)
data["time_key"] = data["Year"] * 12 + data["Month"]

Get unique locations and time points

In [ ]:
locations = data["location_id"].unique()
time_points = sorted(data["time_key"].unique())

In [ ]:
print(f"Number of unique locations: {len(locations)}")
print(f"Number of time points: {len(time_points)}")

Create a dictionary mapping location to index for faster lookup

In [ ]:
loc_to_idx = {loc: idx for idx, loc in enumerate(locations)}

Create a matrix where rows=time, columns=locations

In [ ]:
crime_matrix = np.zeros((len(time_points), len(locations)))

Fill the matrix with crime counts

In [ ]:
for _, row in data.iterrows():
    time_idx = list(time_points).index(row["time_key"])
    loc_idx = loc_to_idx[row["location_id"]]
    crime_matrix[time_idx, loc_idx] = row["crime_count"]

Normalize data

In [ ]:
scaler = MinMaxScaler()
crime_matrix_scaled = scaler.fit_transform(crime_matrix)

Set sequence length (use 12 months to predict the next month)

In [ ]:
seq_length = 12

In this approach:<br>
- Each time step is processed one at a time (no batching of sequences)<br>
- All locations are processed simultaneously (as if they're a "batch")

This means we need the data shaped as:<br>
[number of sequences, sequence_length, number of locations]<br>
where each sequence is a sliding window of 12 months

In [ ]:
def create_sequences(data, seq_length):
    """
    Create sequences from the time series data without batching.
    Each sequence will contain all locations.
    """
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        # Extract sequence of length seq_length
        seq = data[i : i + seq_length]
        # Target is the next time step after the sequence
        target = data[i + seq_length]
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

Create sequences

In [ ]:
X, y = create_sequences(crime_matrix_scaled, seq_length)

In [ ]:
print(f"Number of sequences: {len(X)}")
print(f"Input shape: {X.shape}")  # [n_sequences, seq_length, n_locations]
print(f"Target shape: {y.shape}")  # [n_sequences, n_locations]

Split data into train and validation sets (80% train, 20% validation)

In [ ]:
train_size = int(0.8 * len(X))
X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]

In [ ]:
print(f"Training sequences: {len(X_train)}")
print(f"Validation sequences: {len(X_val)}")

Convert to PyTorch tensors

In [ ]:
X_train_tensor = torch.FloatTensor(X_train).to(device)
y_train_tensor = torch.FloatTensor(y_train).to(device)
X_val_tensor = torch.FloatTensor(X_val).to(device)
y_val_tensor = torch.FloatTensor(y_val).to(device)

Define the RNN model

In [ ]:
class CrimeRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(CrimeRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Define the GRU layer
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)

        # Define the output layer
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x, h=None):
        # Initialize hidden state if not provided
        if h is None:
            h = torch.zeros(self.num_layers, 1, self.hidden_size).to(device)
        else:
            # Detach the hidden state to prevent backpropagation through the entire history
            h = h.detach()

        # Forward propagate the GRU
        # Input shape: [1, seq_length, n_locations]
        # Output shape: [1, seq_length, hidden_size]
        out, h = self.gru(x.unsqueeze(0), h)

        # Decode the hidden state of the last time step
        # out[:, -1, :] shape: [1, hidden_size]
        # Output shape after linear layer: [1, n_locations]
        out = self.fc(out[:, -1, :])
        return out.squeeze(0), h

Initialize model parameters

In [ ]:
input_size = len(locations)  # Number of locations
hidden_size = 32
num_layers = 2
output_size = len(locations)  # Predicting crime count for all locations

Initialize the model

In [ ]:
model = CrimeRNN(input_size, hidden_size, num_layers, output_size).to(device)
print(model)

Define loss function and optimizer

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

Training function

In [ ]:
def train_model(
    model, X_train, y_train, X_val, y_val, criterion, optimizer, num_epochs=50
):
    train_losses = []
    val_losses = []
    start_time = time.time()

    # No batches - process each sequence individually
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        h = None  # Reset hidden state at the start of each epoch

        # Process each sequence (no batching)
        for i in range(len(X_train)):
            # Get a single sequence with all locations
            sequence = X_train[i]
            target = y_train[i]

            # Forward pass
            output, h = model(sequence, h)
            loss = criterion(output, target)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            h = None  # Reset hidden state for validation
            for i in range(len(X_val)):
                output, h = model(X_val[i], h)
                loss = criterion(output, y_val[i])
                val_loss += loss.item()

        # Calculate average losses
        train_loss /= len(X_train)
        val_loss /= len(X_val)
        train_losses.append(train_loss)
        val_losses.append(val_loss)

        # Print progress
        if (epoch + 1) % 5 == 0:
            elapsed = time.time() - start_time
            print(
                f"Epoch [{epoch+1}/{num_epochs}], "
                f"Train Loss: {train_loss:.6f}, "
                f"Val Loss: {val_loss:.6f}, "
                f"Time: {elapsed:.2f}s"
            )
    return train_losses, val_losses

Train the model

In [ ]:
num_epochs = 20
train_losses, val_losses = train_model(
    model,
    X_train_tensor,
    y_train_tensor,
    X_val_tensor,
    y_val_tensor,
    criterion,
    optimizer,
    num_epochs,
)

Plot training and validation loss

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label="Training Loss")
plt.plot(val_losses, label="Validation Loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.savefig("training_validation_loss.pdf")
plt.show()

Function to predict future values

In [ ]:
def predict_future(model, initial_sequence, num_predictions=12):
    model.eval()

    # Start with the last known sequence
    current_sequence = initial_sequence.clone()
    predictions = []
    h = None
    with torch.no_grad():
        for _ in range(num_predictions):
            # Get prediction for next month
            pred, h = model(current_sequence, h)
            predictions.append(pred.cpu().numpy())

            # Update sequence for next prediction (remove oldest, add prediction)
            current_sequence = torch.cat(
                [
                    current_sequence[1:],  # Remove the first time step
                    pred.unsqueeze(0),  # Add the prediction as the last time step
                ],
                dim=0,
            )
    return np.array(predictions)

Get the last sequence from the data for prediction

In [ ]:
last_sequence = torch.FloatTensor(crime_matrix_scaled[-seq_length:]).to(device)

Make predictions for the next 12 months

In [ ]:
num_future_months = 12
future_scaled = predict_future(model, last_sequence, num_future_months)

Inverse transform the predictions to get actual crime counts

In [ ]:
future_predictions = scaler.inverse_transform(future_scaled)

Create dataframe with predictions

In [ ]:
prediction_df = pd.DataFrame()

Generate date range for future predictions

In [ ]:
last_date = pd.to_datetime(f"{time_points[-1]//12}-{time_points[-1]%12+1}-01")
future_dates = pd.date_range(start=last_date, periods=num_future_months + 1, freq="M")[
    1:
]

Add predictions for each location to the dataframe

In [ ]:
for i, loc in enumerate(locations):
    prediction_df[loc] = [max(0, round(val)) for val in future_predictions[:, i]]

In [ ]:
prediction_df.index = future_dates
prediction_df.index.name = "Date"

Display sample of predictions

In [ ]:
print("\nSample predictions for the next 12 months:")
print(prediction_df.iloc[:, :5].head())  # Show first 5 locations, first 5 months

Calculate total crime counts per month

In [ ]:
prediction_df["Total"] = prediction_df.sum(axis=1)
print("\nPredicted total crime counts per month:")
print(prediction_df["Total"])

Plot predictions for a few locations

In [ ]:
plt.figure(figsize=(15, 10))
num_plots = 5

Find locations with significant activity

In [ ]:
active_locations = []
for i in range(len(locations)):
    if np.max(crime_matrix[:, i]) > 5:  # Locations with at least some activity
        active_locations.append(i)
        if len(active_locations) >= num_plots:
            break

If not enough active locations found, just take the first few

In [ ]:
if len(active_locations) < num_plots:
    active_locations = list(range(min(num_plots, len(locations))))

Plot each location

In [ ]:
for i, loc_idx in enumerate(active_locations):
    loc = locations[loc_idx]

    # Historical data
    historical = crime_matrix[:, loc_idx]

    # Time points for x-axis
    time_indices = np.arange(len(historical) + num_future_months)
    plt.subplot(num_plots, 1, i + 1)

    # Plot historical data
    plt.plot(
        time_indices[: len(historical)], historical, label="Historical", color="blue"
    )

    # Plot predictions
    plt.plot(
        time_indices[len(historical) :],
        future_predictions[:, loc_idx],
        label="Predicted",
        color="red",
        linestyle="--",
    )

    # Add a vertical line separating historical and predicted data
    plt.axvline(x=len(historical) - 1, color="gray", linestyle="--")
    plt.title(f"Crime Count for {loc}")
    plt.legend()
    plt.grid(True)

In [ ]:
plt.tight_layout()
plt.savefig("location_predictions.pdf")
plt.show()

Export predictions to CSV

In [ ]:
prediction_df.to_csv("crime_predictions.csv")
print("\nPredictions saved to 'crime_predictions.csv'")

Save the model

In [ ]:
torch.save(
    {
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "scaler": scaler,
        "locations": locations,
    },
    "crime_prediction_model.pth",
)

In [ ]:
print("Model saved to 'crime_prediction_model.pth'")